In [1]:
# this chemkin file is from the cti generated by rmg

# load a bunch of stuff
from __future__ import division
# load
import cantera as ct
import numpy as np
import scipy
import pylab
import matplotlib
import matplotlib.pyplot  as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import NullFormatter, MaxNLocator, LogLocator
import csv
from pydas.dassl import DASSL
import os
import rmgpy
import rmg
import re
import operator
import pandas as pd
%matplotlib inline

gas = ct.Solution('./chem_annotated.cti','gas')
surf = ct.Interface('./chem_annotated.cti','surface1', [gas])
print "This mechanism contains %d gas reactions and %d surface reactions"%(gas.n_reactions, surf.n_reactions)
i_n2 = gas.species_index('N2')
i_ch4 = gas.species_index('CH4(2)')
i_o2 = gas.species_index('O2(3)')
i_co2 = gas.species_index('CO2(4)')
i_h2o = gas.species_index('H2O(5)')
i_h2 = gas.species_index('H2(6)')
i_co = gas.species_index('CO(7)')

/Users/emilymazeau/.local/lib/python2.7/site-packages/scipy/sparse/lil.py:19: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _csparsetools
/Users/emilymazeau/.local/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:165: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._shortest_path import shortest_path, floyd_warshall, dijkstra,\
/Users/emilymazeau/.local/lib/python2.7/site-packages/scipy/sparse/csgraph/_validation.py:5: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._tools import csgraph_to_dense, csgraph_from_dense,\
/Users/emilymazeau/.local/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:167: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._traversal import breadth_first_order, depth_first_order, \
/Users/emilymazeau

/Users/emilymazeau/anaconda2/envs/rmg_env/lib/python2.7/site-packages/pandas/core/window.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  import pandas._libs.window as _window
/Users/emilymazeau/anaconda2/envs/rmg_env/lib/python2.7/site-packages/pandas/core/groupby.py:68: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import lib, groupby as libgroupby, Timestamp, NaT, iNaT
/Users/emilymazeau/anaconda2/envs/rmg_env/lib/python2.7/site-packages/pandas/core/reshape/reshape.py:31: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos as _algos, reshape as _reshape
/Users/emilymazeau/anaconda2/envs/rmg_env/lib/python2.7/site-packages/pandas/io/parsers.py:45: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  import pandas._libs.parsers a

This mechanism contains 123 gas reactions and 87 surface reactions


In [2]:
#######################################################################
# Input Parameters
#######################################################################

volume = 1.0
cat_area_per_vol = 1.0E2 #I made this up!

cat_area = cat_area_per_vol * volume

#initial conditions
T0 = 1000.
P0 = 1.0 #bar
#initial composition (gas mol fraction)
CH4_0 = 0.1
O2_0 = 0.1
N2_0 = 0.8
CO2_0 = 0.

gas.TPX = T0, P0 * ct.one_atm, {'CH4(2)':CH4_0, 'O2(3)':O2_0, 'N2':N2_0, 'CO2(4)':CO2_0}
          #Temp, Pressure, mole fraction
surf.TP = T0, P0 * ct.one_atm
surf.coverages = {'X(1)':1.0}
#surf.coverages = {'PT(S)':0.9, 'O(S)':0.1}

r = ct.IdealGasConstPressureReactor(gas, energy='off')
r.volume = volume
rsurf = ct.ReactorSurface(surf, r, A=cat_area) # A=cat_area

sim = ct.ReactorNet([r])
sim.max_err_test_fails = 12

# set relative and absolute tolerances on the simulation
sim.rtol = 1.0e-12
sim.atol = 1.0e-20

rxn_time = np.linspace(1e-11, 1e3, 100001) #
#rxn_time = np.logspace(-11, 3, 10001) #from 0s to 1000s, log spacing

gas_mole_fracs = np.zeros([gas.n_species, len(rxn_time)])
surf_site_fracs = np.zeros([surf.n_species, len(rxn_time)])
temperature = np.zeros(len(rxn_time))    
pressure = np.zeros(len(rxn_time))   

for i in range(len(rxn_time)):
    time = rxn_time[i] #define time in the reactor
    sim.advance(time) #Advance the simulation to next set time
    temperature[i] = gas.T
    pressure[i] = gas.P/ct.one_atm
    gas_mole_fracs[:,i] = gas.X #
    surf_site_fracs[:,i] = surf.coverages #
    if not i % 500:
        print('  {0:10e}  {1:10f}  {2:10f}  {3:10f}'.format(time, *gas['CH4(2)','O2(3)','N2'].X))



#sim.advance_to_steady_state()
#print('  {0:10e}  {1:10f}  {2:10f}  {3:10f}'.format(sim.time, *gas['CH4','H2','CO'].X))    
    
# CH4_ref = gas_mole_fracs[i_ch4,:]

# setting new sensitivity to be based on time to consume 95% of the o2
zipped = zip(gas_mole_fracs[i_o2,:],rxn_time)
zipped = [list(z) for z in zipped]
for z in zipped:
    if z[0] <= O2_0 * .05:
        O2_ref_time = z[1]
        break

#Plot out simulations results
fig = pylab.figure(dpi=300,figsize=(8,8))
gs = gridspec.GridSpec(2, 1)
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1])

y_min = 1E-3

for i in range(gas.n_species):
    if i != i_n2:
        if np.max(gas_mole_fracs[i,:]) > y_min:
            ax0.loglog(rxn_time, gas_mole_fracs[i,:], label=gas.species_name(i) )

for i in range(surf.n_species):
    if np.max(surf_site_fracs[i,:]) > y_min:
        ax1.loglog(rxn_time, surf_site_fracs[i,:], label=surf.species_name(i) )
            
ax0.legend(loc='lower left', fontsize = 12)
ax1.legend(loc='lower left', fontsize = 12)

ax0.set_ylabel("gas-phase mole fraction")
ax0.set_ylim(y_min,0.5)
ax0.set_xlim(1.0E-4,max(rxn_time))
ax1.set_xlim(1.0E-4,max(rxn_time))
ax0.set_xlabel("Time(s)")
ax1.set_ylabel("surface site fraction")
#ax0.xaxis.set_major_locator(MaxNLocator(6))
#ax0.yaxis.set_major_locator(LogLocator(base=10.0, numticks=3))
#ax0.tick_params(axis='both', which='major', labelsize=10)
ax1.set_ylim(y_min, 1.1 )
ax1.set_xlabel("Time(s)")


#######################
##   FLUX DIAGRAMS   ##
#######################
# steps = len(temperature)
# for i in range(0, steps, steps//20):
#     gas.TP = temperature[i], pressure[i] * ct.one_atm
#     gas.X  = gas_mole_fracs[:,i]
#     surf.TP = temperature[i], pressure[i] * ct.one_atm
#     surf.coverages = surf_site_fracs[:,i]
    
#     element = 'O'

#     diagram = ct.ReactionPathDiagram(surf, element)
#     diagram.title = 'Reaction path diagram following {0}'.format(element)
#     diagram.label_threshold = 0.000001

#     dot_file = 'rxnpathO{:06d}.dot'.format(i)
#     img_file = 'rxnpathO{:06d}.png'.format(i)
#     img_path = os.path.join(os.getcwd(), img_file)

#     diagram.write_dot(dot_file)
# #     print(diagram.get_data())

#     print("Wrote graphviz input file to '{0}'.".format(os.path.join(os.getcwd(), dot_file)))

#     os.system('dot {0} -Tpng -o{1} -Gdpi=200'.format(dot_file, img_file))
#     print("Wrote graphviz output file to '{0}'.".format(img_path))

#     from IPython.display import Image
#     Image(filename=img_file) 
    
# for i in range(0, steps, steps//20):
#     gas.TP = temperature[i], pressure[i] * ct.one_atm
#     gas.X  = gas_mole_fracs[:,i]
#     surf.TP = temperature[i], pressure[i] * ct.one_atm
#     surf.coverages = surf_site_fracs[:,i]
    
#     element = 'C'

#     diagram = ct.ReactionPathDiagram(surf, element)
#     diagram.title = 'Reaction path diagram following {0}'.format(element)
#     diagram.label_threshold = 0.000001

#     dot_file = 'rxnpathC{:06d}.dot'.format(i)
#     img_file = 'rxnpathC{:06d}.png'.format(i)
#     img_path = os.path.join(os.getcwd(), img_file)

#     diagram.write_dot(dot_file)
# #     print(diagram.get_data())

#     print("Wrote graphviz input file to '{0}'.".format(os.path.join(os.getcwd(), dot_file)))

#     os.system('dot {0} -Tpng -o{1} -Gdpi=200'.format(dot_file, img_file))
#     print("Wrote graphviz output file to '{0}'.".format(img_path))

#     from IPython.display import Image
#     Image(filename=img_file) 


  1.000000e-11    0.100000    0.100000    0.800000
  5.000000e+00    0.098557    0.098764    0.799382
  1.000000e+01    0.092738    0.093280    0.798247
  1.500000e+01    0.086823    0.087448    0.797191
  2.000000e+01    0.081530    0.082091    0.796288
  2.500000e+01    0.076854    0.077276    0.795530
  3.000000e+01    0.072717    0.072960    0.794889
  3.500000e+01    0.069038    0.069081    0.794340
  4.000000e+01    0.065745    0.065578    0.793866
  4.500000e+01    0.062781    0.062400    0.793454
  5.000000e+01    0.060099    0.059502    0.793093
  5.500000e+01    0.057658    0.056847    0.792775
  6.000000e+01    0.055428    0.054405    0.792495
  6.500000e+01    0.053382    0.052149    0.792248
  7.000000e+01    0.051498    0.050058    0.792028
  7.500000e+01    0.049757    0.048114    0.791833
  8.000000e+01    0.048143    0.046300    0.791660
  8.500000e+01    0.046643    0.044604    0.791506
  9.000000e+01    0.045245    0.043013    0.791369
  9.500000e+01    0.043940    0

CanteraError: 
***********************************************************************
CanteraError thrown by CVodesIntegrator::integrate:
CVodes error encountered. Error code: -10
At t = 777.986 repeated recoverable right-hand side function errors.

Exceptions caught during RHS evaluation:
ydot contains non-finite elements:

ydot[0] = nan
ydot[2] = nan
ydot[3] = nan
ydot[4] = nan
ydot[5] = nan
ydot[6] = nan
ydot[7] = nan
ydot[8] = nan
ydot[9] = nan
ydot[10] = nan
ydot[11] = nan
ydot[12] = nan
ydot[13] = nan
ydot[14] = nan
ydot[15] = nan
ydot[16] = nan
ydot[17] = nan
ydot[18] = nan
ydot[19] = nan
ydot[20] = nan
ydot[21] = nan
ydot[22] = nan
ydot[23] = nan
ydot[24] = nan
ydot[25] = nan
ydot[26] = nan
ydot[27] = nan
ydot[28] = nan
ydot[29] = nan
ydot[30] = inf
ydot[31] = nan
ydot[32] = nan
ydot[33] = nan
ydot[34] = -inf
ydot[35] = inf
ydot[36] = -inf
ydot[37] = nan
ydot[38] = nan
ydot[39] = nan
ydot[40] = -inf
ydot[41] = -inf
ydot[43] = -inf
ydot[44] = nan
ydot[45] = -inf
ydot[46] = nan


In [ ]:
with open('time.txt','w') as f:
    f.write(str(O2_ref_time))

plt.semilogy(gas.forward_rates_of_progress,'.')

for i in gas.forward_rates_of_progress.argsort()[:-5:-1]:
    print i, gas.reaction_equation(i)

plt.semilogy(gas.reverse_rates_of_progress,'.')

for i in gas.reverse_rates_of_progress.argsort()[:-5:-1]:
    print i, gas.reaction_equation(i)

plt.semilogy(surf.forward_rates_of_progress,'.')

for i in surf.forward_rates_of_progress.argsort()[:-5:-1]:
    print i, surf.reaction_equation(i)

plt.semilogy(surf.reverse_rates_of_progress,'.')

for i in surf.reverse_rates_of_progress.argsort()[:-5:-1]:
    print i, surf.reaction_equation(i)

In [4]:
# Set the value of the perturbation
dk = 1.0e-2
# create the array to hold the sensitivty at each time step
Sensitivity = np.zeros([surf.n_reactions,len(rxn_time)])

rxns = []
sens = []

#cycle through each reaction
for m in range(surf.n_reactions):
    surf.set_multiplier(1.0) # reset all multipliers  
    surf.set_multiplier(1+dk, m) # perturb reaction m
    
    gas.TPX = T0, P0 * ct.one_atm, {'CH4(2)':CH4_0, 'O2(3)':O2_0, 'N2':N2_0}
              #Temp, Pressure, mole fraction
    surf.TP = T0, P0 * ct.one_atm
    surf.coverages = {'X(1)':1.0}
    
    r = ct.IdealGasConstPressureReactor(gas, energy='off')
    r.volume = volume
    rsurf = ct.ReactorSurface(surf, r, A=cat_area) # A=cat_area


    sim = ct.ReactorNet([r])
    sim.max_err_test_fails = 12


    # set relative and absolute tolerances on the simulation
    sim.rtol = 1.0e-12
    sim.atol = 1.0e-20
    gas_mole_fracs = np.zeros([gas.n_species, len(rxn_time)])
    temperature = np.zeros(len(rxn_time))    
    pressure = np.zeros(len(rxn_time))   

    for i in range(len(rxn_time)):
        time = rxn_time[i] #define time in the reactor
        sim.advance(time) #Advance the simulation to next set time
        temperature[i] = gas.T
        pressure[i] = gas.P/ct.one_atm
        gas_mole_fracs[:,i] = gas.X #[mol.frac,nth reactor]
    # here is the new output
#     CH4_new = gas_mole_fracs[i_ch4,:]

    O2_new = gas_mole_fracs[i_o2,:]
    zipped = zip(gas_mole_fracs[i_o2,:],rxn_time)
    zipped = [list(z) for z in zipped]
    for z in zipped:
        if z[0] <= O2_0 * .05:
            O2_new_time = z[1]
            break

#     Sens = (CH4_new-CH4_ref)/(CH4_ref*dk)
    Sens = (O2_ref_time - O2_new_time)/(O2_ref_time*dk) # positive if it speeds up the reaction
#     max_sensitivity = max(max(Sens),-min(Sens))
#    if (1==0):
#        if (max_sensitivity>0.01):
#            print "%d %s %.2F"%( m, surf.reaction_equations()[m],  max_sensitivity)
#    else:
#     print "%d %s %.2F"%( m, surf.reaction_equations()[m],  max_sensitivity)
    print "%d %s %.2F"%( m, surf.reaction_equations()[m],  Sens)
#     Sensitivity[m,:] = Sens
    rxns.append(surf.reaction_equations()[m])
    sens.append(Sens)
surf.set_multiplier(1.0)

 

0 CH4(2) + X(1) <=> CH4X(33) 0.00
1 O2(3) + 2 X(1) <=> 2 OX(20) 3.95
2 C2H6(8) + 2 X(1) <=> 2 CH3X(22) 0.00
3 CH3(10) + X(1) <=> CH3X(22) 0.00
4 H2(6) + 2 X(1) <=> 2 HX(19) 0.00
5 CH3X(22) + HX(19) <=> CH4(2) + 2 X(1) -3.11
6 H(12) + X(1) <=> HX(19) 0.00
7 CH2X(134) + HX(19) <=> CH3X(22) + X(1) 0.00
8 CHX(26) + HX(19) <=> CH2X(134) + X(1) 0.00
9 2 CH2X(134) <=> CH3X(22) + CHX(26) 0.00
10 CH3X(22) + HOX(23) <=> CH4(2) + OX(20) + X(1) 0.64
11 HOX(23) + HX(19) <=> H2O(5) + 2 X(1) -1.95
12 2 HOX(23) <=> H2O(5) + OX(20) + X(1) 0.00
13 CH3OH(14) + 2 X(1) <=> CH3X(22) + HOX(23) 0.00
14 OH(17) + X(1) <=> HOX(23) 0.00
15 CH2X(134) + HOX(23) <=> CH3X(22) + OX(20) 0.00
16 HOX(23) + X(1) <=> HX(19) + OX(20) 0.00
17 CHX(26) + HOX(23) <=> CH2X(134) + OX(20) 0.00
18 H2(6) + X(1) <=> H2X(36) 0.00
19 H2O(5) + X(1) <=> H2OX(35) 0.00
20 CX(24) + HX(19) <=> CHX(26) + X(1) 0.00
21 CX(24) + HOX(23) <=> CHX(26) + OX(20) 0.00
22 CH3X(22) + CX(24) <=> CH2X(134) + CHX(26) 0.00
23 CH2X(134) + CX(24) <=> 2 CHX(26

In [5]:
#Plot out simulations results
# fig = pylab.figure(dpi=300,figsize=(8,8))
# gs = gridspec.GridSpec(1, 1)
# ax0 = plt.subplot(gs[0])


# for m in range(surf.n_reactions):
#     y_max = 0.00001
#     if (1==1):
#         if max(Sensitivity[m,:])>y_max or min(Sensitivity[m,:])<-y_max:
#             lab = surf.reaction_equations()[m]
#             ax0.semilogx(rxn_time, Sensitivity[m,:], label=lab )
#     else:
#         if max(Normed_sensitivity[m,:])>y_max or min(Normed_sensitivity[m,:])<-y_max:
#             lab = surf.reaction_equations()[m]
#             ax0.semilogx(rxn_time, Normed_sensitivity[m,:], label=lab )
        
# ax0.legend(loc='best', fontsize = 12)   
# ax0.set_xlim(rxn_time[1], rxn_time[-1] )

# ax0.set_xlabel("Time(s)")
# ax0.set_ylabel("Sensitivity Coefficient")

In [6]:
# # now integrate with respect to time
# from numpy import trapz

# # for m in range(surf.n_reactions):
# #     ans = trapz(Sensitivity[m,:], rxn_time)#/max(rxn_time)
# #     if ans>1E-4:
# #         print "%d %s %.5f"%( m, surf.reaction_equations()[m], ans )

# n = []
# rxns = []
# for m in range(surf.n_reactions):
#     n.append(trapz(Sensitivity[m,:], rxn_time))#/max(rxn_time)
#     rxns.append(surf.reaction_equations()[m])

# ans = dict(zip(rxns,n))
# sorted_ans = sorted(ans.items(), key=operator.itemgetter(1), reverse=True)

# for x in sorted_ans:
#     print '%s %.4f'%(x)

In [7]:
species_dict = rmgpy.data.kinetics.KineticsLibrary().getSpecies('species_dictionary.txt')
keys = species_dict.keys()

# get the first listed smiles string for each molecule
smile = []
for s in species_dict:
    smile.append(species_dict[s].molecule[0])
    if len(species_dict[s].molecule) is not 1:
        print 'There are %d dupllicate smiles for %s:'%(len(species_dict[s].molecule),s)
        for a in range(len(species_dict[s].molecule)):
            print '%s'%(species_dict[s].molecule[a])
            
# translate the molecules from above into just smiles strings
smiles = []
for s in smile:
    smiles.append(s.toSMILES())
    
names = dict(zip(keys,smiles))

rxns_translated = []

# now replace each molecule with smiles
for x in rxns:
    for key, smile in names.iteritems():
        x = re.sub(re.escape(key), smile, x)
    rxns_translated.append(x)

answer = dict(zip(rxns_translated, sens))
sorted_answer = sorted(answer.items(), key=operator.itemgetter(1), reverse=True)
# for x in sorted_answer:
#     print '%s %.4f'%(x)

In [8]:
# write to csv file
(pd.DataFrame.from_dict(data=sorted_answer, orient='columns')
.to_csv('dict.csv', header=False))
